# Trials Generation Script

## Loads all stimuli details from a CSV and generates trial sets

### Key features of each stimulus are also included in dataframe rows (CSV rows). 
 
Raúl Arrabales - Change Blindness in Hierarchical Stimuli Project

In [125]:
from os import listdir
from os.path import isfile, join

import pandas as pd

In [126]:
# Load dataframe with all stimuli
sdf = pd.read_csv('AllStimuli.csv',sep=';')

In [127]:
# Stimuli table
sdf.head(10)

,StimulusId,FileName,Pattern,G1,L1,G2,L2,G3,L3,G4,L4
0,0,1Row-Xf-Xd-Xd-Xf,1Row,X,f,X,d,X,d,X,f
1,1,1Row-Xd-Xg-Xd-Xg,1Row,X,d,X,g,X,d,X,g
2,2,1Row-Xd-Xf-Xa-Xg,1Row,X,d,X,f,X,a,X,g
3,3,1Row-Dx-Bg-Ef-Hd,1Row,D,x,B,g,E,f,H,d
4,4,1Row-Gl-Bg-Ah-Hd,1Row,G,l,B,g,A,h,H,d
5,5,1Row-Xf-Xg-Xd-Xg,1Row,X,f,X,g,X,d,X,g
6,6,1Row-Hc-Rd-Xf-Ah,1Row,H,c,R,d,X,f,A,h
7,7,1Row-Ae-Xd-Rx-Hf,1Row,A,e,X,d,R,x,H,f
8,8,1Row-Gh-Bg-Eh-Hd,1Row,G,h,B,g,E,h,H,d
9,9,1Row-Hc-Bh-Xf-Ah,1Row,H,c,B,h,X,f,A,h


In [128]:
# Stimuli table stats
sdf.describe()

,StimulusId
count,59.000000
mean,29.000000
std,17.175564
min,0.000000
25%,14.500000
50%,29.000000
75%,43.500000
max,58.000000


In [129]:
# Dataframe to store all trials
trial_columns = [
    'TrialId',     # Trial ID
    'S1',          # Stimulus 1
    'S2',          # Stimulus 2
    'ISI',         # ISI (inter-stimulus interval) is ms
    'ChangeType',  # Type of change (L:local, G:global, M:local and global, N:no change, Z: Multiple changes)
    'Correct']     # Correct key to press (L, G, N, M, Z)
trials_df = pd.DataFrame(columns=trial_columns)
trials_df.head()

,TrialId,S1,S2,ISI,ChangeType,Correct


In [130]:
# Number of local changes between two stimuli
def LocalChanges( S1, S2 ):
    numLocalChanges = 0
    
    if S1['L1'] != S2['L1']:
        numLocalChanges +=1
    if S1['L2'] != S2['L2']:
        numLocalChanges +=1
    if S1['L3'] != S2['L3']:
        numLocalChanges +=1
    if S1['L4'] != S2['L4']:
        numLocalChanges +=1    
        
    return numLocalChanges


In [131]:
# Number of global changes between two stimuli
def GlobalChanges( S1, S2 ):
    numGlobalChanges = 0
    
    if S1['G1'] != S2['G1']:
        numGlobalChanges +=1
    if S1['G2'] != S2['G2']:
        numGlobalChanges +=1
    if S1['G3'] != S2['G3']:
        numGlobalChanges +=1
    if S1['G4'] != S2['G4']:
        numGlobalChanges +=1 
        
    return numGlobalChanges


In [132]:
# Given one local and one global change, True if both changes are in the same letter (position)
def MChange( S1, S2):
    posLocal = 0
    posGlobal = 0
    
    if S1['L1'] != S2['L1']:
        posLocal = 1
    elif S1['L2'] != S2['L2']:
        posLocal = 2
    elif S1['L3'] != S2['L3']:
        posLocal = 3
    elif S1['L4'] != S2['L4']:
        posLocal = 4
        
    if S1['G1'] != S2['G1']:
        posGlobal = 1
    if S1['G2'] != S2['G2']:
        posGlobal = 2
    if S1['G3'] != S2['G3']:
        posGlobal = 3
    if S1['G4'] != S2['G4']:
        posGlobal = 4
        
    return posLocal == posGlobal
    

In [135]:
# Iterate over all rows combined with all (same) rows - NxN combinations
trials_df = pd.DataFrame(columns=trial_columns) # Reset trials df
i = 0
ISI = 250
print('Processing...')
for index1, row1 in sdf.iterrows():
    for index2, row2 in sdf.iterrows():
        # print('Processing trial Id '+str(i)+': S'+str(index1)+' and S'+str(index2) )
        i += 1
        
        # N - No Change from S1 to S2
        if index1 == index2: 
            change = 'N' # Same stimulus means no change
            trials_df.loc[len(trials_df)] = [str(i), row1['FileName'], row2['FileName'], str(ISI), change, change]
            
        # L - One Local Change from S1 to S2
        elif LocalChanges(row1, row2) == 1 and GlobalChanges(row1, row2) == 0:
            change = 'L'
            trials_df.loc[len(trials_df)] = [str(i), row1['FileName'], row2['FileName'], str(ISI), change, change]
        
        # G - One Global Change from S1 to S2
        elif LocalChanges(row1, row2) == 0 and GlobalChanges(row1, row2) == 1:
            change = 'G'
            trials_df.loc[len(trials_df)] = [str(i), row1['FileName'], row2['FileName'], str(ISI), change, change]
                        
        # M - One Local and One Global Change from S1 to S2
        elif LocalChanges(row1, row2) == 1 and GlobalChanges(row1, row2) == 1 and MChange(row1, row2):
            change = 'M'
            trials_df.loc[len(trials_df)] = [str(i), row1['FileName'], row2['FileName'], str(ISI), change, change]
            
        # Z - More than one change in same letter
        else:
            change = 'Z'
            trials_df.loc[len(trials_df)] = [str(i), row1['FileName'], row2['FileName'], str(ISI), change, change]

print('Done. Processed: ' +str(i)+ ' trials.')
        
                   

Processing...
Done. Processed: 3481 trials.


In [136]:
print('Total number of trials generated: ' + str(len(trials_df)))

Total number of trials generated: 3481


In [137]:
# Trials Dataframe samlple: 
trials_df.head(10)

,TrialId,S1,S2,ISI,ChangeType,Correct
0,1,1Row-Xf-Xd-Xd-Xf,1Row-Xf-Xd-Xd-Xf,250,N,N
1,2,1Row-Xf-Xd-Xd-Xf,1Row-Xd-Xg-Xd-Xg,250,Z,Z
2,3,1Row-Xf-Xd-Xd-Xf,1Row-Xd-Xf-Xa-Xg,250,Z,Z
3,4,1Row-Xf-Xd-Xd-Xf,1Row-Dx-Bg-Ef-Hd,250,Z,Z
4,5,1Row-Xf-Xd-Xd-Xf,1Row-Gl-Bg-Ah-Hd,250,Z,Z
5,6,1Row-Xf-Xd-Xd-Xf,1Row-Xf-Xg-Xd-Xg,250,Z,Z
6,7,1Row-Xf-Xd-Xd-Xf,1Row-Hc-Rd-Xf-Ah,250,Z,Z
7,8,1Row-Xf-Xd-Xd-Xf,1Row-Ae-Xd-Rx-Hf,250,Z,Z
8,9,1Row-Xf-Xd-Xd-Xf,1Row-Gh-Bg-Eh-Hd,250,Z,Z
9,10,1Row-Xf-Xd-Xd-Xf,1Row-Hc-Bh-Xf-Ah,250,Z,Z


In [138]:
# Trials with no change from S1 to S2
N_trials = trials_df[trials_df['ChangeType'] == 'N']
print(str(len(N_trials)) + ' trials with no change:')
N_trials.head()

59 trials with no change:


,TrialId,S1,S2,ISI,ChangeType,Correct
0,1,1Row-Xf-Xd-Xd-Xf,1Row-Xf-Xd-Xd-Xf,250,N,N
60,61,1Row-Xd-Xg-Xd-Xg,1Row-Xd-Xg-Xd-Xg,250,N,N
120,121,1Row-Xd-Xf-Xa-Xg,1Row-Xd-Xf-Xa-Xg,250,N,N
180,181,1Row-Dx-Bg-Ef-Hd,1Row-Dx-Bg-Ef-Hd,250,N,N
240,241,1Row-Gl-Bg-Ah-Hd,1Row-Gl-Bg-Ah-Hd,250,N,N


In [139]:
# Trials with one local change from S1 to S2
L_trials = trials_df[trials_df['ChangeType'] == 'L']
print(str(len(L_trials)) + ' trials with one local change (L):')
L_trials.head()

124 trials with one local change (L):


,TrialId,S1,S2,ISI,ChangeType,Correct
20,21,1Row-Xf-Xd-Xd-Xf,1Row-Xf-Xd-Xd-Xg,250,L,L
64,65,1Row-Xd-Xg-Xd-Xg,1Row-Xf-Xg-Xd-Xg,250,L,L
69,70,1Row-Xd-Xg-Xd-Xg,1Row-Xd-Xg-Xg-Xg,250,L,L
85,86,1Row-Xd-Xg-Xd-Xg,1Row-Xd-Xf-Xd-Xg,250,L,L
113,114,1Row-Xd-Xg-Xd-Xg,1Row-Xd-Xd-Xd-Xg,250,L,L


In [141]:
# Trials with one global change from S1 to S2
G_trials = trials_df[trials_df['ChangeType'] == 'G']
print(str(len(G_trials)) + ' trials with one global change (G):')
G_trials.head()

8 trials with one global change (G):


,TrialId,S1,S2,ISI,ChangeType,Correct
286,287,1Row-Gl-Bg-Ah-Hd,1Row-Al-Bg-Ah-Hd,250,G,G
483,484,1Row-Gh-Bg-Eh-Hd,1Row-Gh-Bg-Ah-Hd,250,G,G
657,658,1Row-Gh-Bg-Ah-Hd,1Row-Gh-Bg-Eh-Hd,250,G,G
990,991,1Row-Xg-Bh-Xg-Bf,1Row-Rg-Bh-Xg-Bf,250,G,G
1093,1094,1Row-Gh-Bg-Ef-Hd,1Row-Dh-Bg-Ef-Hd,250,G,G


In [142]:
# Trials with one global and one local change from S1 to S2
M_trials = trials_df[trials_df['ChangeType'] == 'M']
print(str(len(M_trials)) + ' trials with one global and local change (M):')
M_trials.head()

46 trials with one global and local change (M):


,TrialId,S1,S2,ISI,ChangeType,Correct
195,196,1Row-Dx-Bg-Ef-Hd,1Row-Gh-Bg-Ef-Hd,250,M,M
232,233,1Row-Dx-Bg-Ef-Hd,1Row-Ef-Bg-Ef-Hd,250,M,M
283,284,1Row-Gl-Bg-Ah-Hd,1Row-Ae-Bg-Ah-Hd,250,M,M
363,364,1Row-Hc-Rd-Xf-Ah,1Row-Hc-Bh-Xf-Ah,250,M,M
537,538,1Row-Hc-Bh-Xf-Ah,1Row-Hc-Rd-Xf-Ah,250,M,M


In [143]:
# Export trials dataframe to a CSV file:
trials_df.to_csv('AllTrials.csv', sep =';', index_label='Id')